## Density Functional Perturbation Theory (DFPT)
---

In this notebook we will move to the perturbation theory calculation of phonon frequencies. This type of calculation is another step up on the scale of computational complexity and demands. Thus, the warnings about *not* following blindly the presented scheme in the real calculations applies here even more. The calculation is modeled after one of the AbInit tutorials.

We will calculate for AsAl:
- Dynamical matrix at $\Gamma$ and X
- Phonon frequencies at $\Gamma$ and X
- Born effective charges
- Dielectric tensor
- LO-TO splitting


### Calculation setup

In [2]:
%pylab inline
import ase
import ase.io
from ase.spacegroup import crystal
from ase.calculators.abinit import Abinit
from ase import units as un
from elastic import get_pressure

# Utility functions
from tqdm.auto import tqdm
import spglib

Populating the interactive namespace from numpy and matplotlib


In [86]:
import subprocess
from ase.calculators.calculator import CalculationFailed

def run_calc(cryst):
    '''
    Run the calculation without reading back any results into cryst.
    '''
    cryst.calc.write_input(cryst, ['energy'], ['positions'])
    cmd = cryst.calc.command.replace('PREFIX', cryst.calc.prefix)
    proc = subprocess.Popen(cmd, shell=True, cwd=cryst.calc.directory)
    errorcode = proc.wait()
    if errorcode:
        path = os.path.abspath(cryst.calc.directory)
        msg = ('Calculator "{}" failed with command "{}" failed in '
               '{} with error code {}'.format(cryst.calc.name, cmd,
                                              path, errorcode))
        raise CalculationFailed(msg)
    return True


def scan_param(cryst, param, start=0, stop=1, steps=7, 
               lista=None, log_scale=True, int_par=False):
    '''
    Scan a range of parameter param and plot 
    '''
    fi = lambda x: x
    fo = lambda x: x

    if lista is None:
        if log_scale :
            fi = log
            fo = exp
        lista = linspace(fi(start), fi(stop), steps)
    else :
        int_par=False
    
    dat = []
    vs = []
        
    for v in lista:
        print(f'{param}={fo(v)}: ', end='')
        if int_par:
            cryst.calc.set(**{param: int(fo(v))})
        else :
            cryst.calc.set(**{param: fo(v)})
        vs.append(fo(v))
        dat.append([cryst.get_potential_energy(),
                    cryst.get_stress()[:3].mean()])
        print(f'{dat[-1][0]:.3f} eV ; {dat[-1][1]/un.GPa:6.3f} GPa',)
    dat = array(dat).T
    return vs, dat

In [123]:
a = 5.61456927
prim = crystal(('As','Al'), basis=((0,0,0), (1/4,1/4,1/4)), 
        spacegroup=216, cellpar=(a, a, a, 90, 90, 90),
        primitive_cell=True)

prim.cell.cellpar()

array([ 3.9701,  3.9701,  3.9701, 60.    , 60.    , 60.    ])

In [19]:
prim.get_scaled_positions()

array([[0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25]])

In [69]:
spglib.get_spacegroup(prim)

'F-43m (216)'

In [14]:
lat = prim.cell.get_bravais_lattice()
print(lat.description())

In [15]:
work_dir = 'work_10'

In [53]:
def create_calculator(label, directory=work_dir, ecut=420):
    return Abinit(label=label, directory=directory, v8_legacy_format=True,              
              ecut= ecut,
              pps = 'pspnc',
              #pps = 'pawxml', pawecutdg = 750, xc = 'GGA', toldfe = 1.0e-6,
              tolvrs=1.0e-18,
              nband=4,
              diemac=9.0,
              nstep=15,
              kptrlatt=[[-4,  4,  4],
                        [ 4, -4,  4],
                        [ 4,  4, -4]]
              )

In [54]:
calc = create_calculator(label='AsAl')
prim.set_calculator(calc)

In [55]:
print(f'  Energy:  {prim.get_potential_energy():.3f} eV')
print( '  Stress: [', 6*' %6.2f' % tuple(prim.get_stress()/un.GPa), '] GPa')
print(f'Pressure:     {get_pressure(prim.get_stress())/un.GPa:.3f} GPa')

  Energy:  -276.122 eV
  Stress: [   -0.07  -0.07  -0.07   0.00   0.00   0.00 ] GPa
Pressure:     0.068 GPa


In [58]:
prim.get_forces()

array([[-0., -0., -0.],
       [-0., -0., -0.]])

### Convergence scan
---
This is a reminder that you should *always* perform convergence analysis of your calculations. We can skip this step by setting `Scan_Corvengence = False` , since we have already performed this analysis in previous part.

In [56]:
Scan_Corvengence = False

In [57]:
if Scan_Corvengence :
    ecut = prim.calc.parameters['ecut']
    ecuts, eres = scan_param(prim, 'ecut', 200, 500)

    figsize(6,6)
    subplot(211)
    plot(ecuts, eres[0], 'o')
    ylabel('Energy (eV)')
    subplot(212)
    plot(ecuts, eres[1]/un.GPa, 'o')
    ylabel('Pressure (GPa)')
    xlabel('ecut (eV)');
    prim.calc.set(ecut=ecut)

### DFPT calculations
---

In [124]:
cr = ase.Atoms(prim)
cr.set_calculator(create_calculator('AsAl_3'))
cr.calc.set(ndtset=4,
            
            #Ground state calculation
            kptopt1=1,              # Automatic generation of k points, taking
                                    # into account the symmetry
            tolvrs1=1.0e-18,        # SCF stopping criterion
            
            #Response Function calculation : d/dk
            rfelfd2=2,              # Activate the calculation of the d/dk perturbation
            rfdir2=[1, 0, 0],       # Need to consider the perturbation in the x-direction only
                                    # This is rather specific, due to the high symmetry of the AlAs crystal
                                    # In general, just use rfdir 1 1 1
                                    # In the present version of ABINIT (v4.6), symmetry cannot be used
                                    # to reduce the number of ddk perturbations
            nqpt2=1,
            qpt2=[0.0, 0.0, 0.0],   # This is a calculation at the Gamma point

            getwfk2=-1,             # Uses as input the output wf of the previous dataset

            kptopt2=2,              # Automatic generation of k points,
                                    # using only the time-reversal symmetry to decrease
                                    # the size of the k point set.

            iscf2=-3,               # The d/dk perturbation must be treated 
                                    # in a non-self-consistent way
            tolwfr2=1.0e-22,        # Must use tolwfr for non-self-consistent calculations
                                    # Here, the value of tolwfr is very low.          
            
            #Response Function calculation : electric field perturbation and phonons
            rfphon3=1,              # Activate the calculation of the atomic dispacement perturbations
            rfatpol3=[1, 2],        # All the atoms will be displaced
            rfelfd3=3,              # Activate the calculation of the electric field perturbation
            rfdir3=[1, 1, 1],       # All directions are selected. However, symmetries will be used to decrease
                                    # the number of perturbations, so only the x electric field is needed
                                    # (and this explains why in the second dataset, rfdir was set to 1 0 0).

            nqpt3=1,
            qpt3=[0.0, 0.0, 0.0],   # This is a calculation at the Gamma point

            getwfk3=-2,             # Uses as input wfs the output wfs of the dataset 1
            getddk3=-1,             # Uses as input ddk wfs the output of the dataset 2

            kptopt3=2,              # Automatic generation of k points,
                                    # using only the time-reversal symmetry to decrease
                                    # the size of the k point set.
            tolvrs3=1.0e-8,
            
            #Response Function calculation : phonon at X
              rfphon4 = 1,            # Activate the calculation of the atomic dispacement perturbations
             rfatpol4 = [1,2],           # All the atoms will be displaced
               rfdir4 = [1,1,1],         # Need all directions but ABINIT will use symmetries if possible

                nqpt4 = 1,
                 qpt4 = [0.0,0.5,0.5],   # This is a calculation at the X point

              getwfk4 = 1,            # Uses as input wfs the output wfs of the dataset 1

              kptopt4 = 3,            # Automatic generation of k points,
                                      # no use of symmetries to decrease 
                                      # the size of the k point set.
              tolvrs4 = 1.0e-8,

           )


{'ndtset': 4,
 'kptopt1': 1,
 'tolvrs1': 1e-18,
 'rfelfd2': 2,
 'rfdir2': [1, 0, 0],
 'nqpt2': 1,
 'qpt2': [0.0, 0.0, 0.0],
 'getwfk2': -1,
 'kptopt2': 2,
 'iscf2': -3,
 'tolwfr2': 1e-22,
 'rfphon3': 1,
 'rfatpol3': [1, 2],
 'rfelfd3': 3,
 'rfdir3': [1, 1, 1],
 'nqpt3': 1,
 'qpt3': [0.0, 0.0, 0.0],
 'getwfk3': -2,
 'getddk3': -1,
 'kptopt3': 2,
 'tolvrs3': 1e-08,
 'rfphon4': 1,
 'rfatpol4': [1, 2],
 'rfdir4': [1, 1, 1],
 'nqpt4': 1,
 'qpt4': [0.0, 0.5, 0.5],
 'getwfk4': 1,
 'kptopt4': 3,
 'tolvrs4': 1e-08}

In [ ]:
run_calc(cr)

### Extract the results
---

The following code is a simple state machine extracting data from the output file.
The code is an ad-hoc type but can be adapted to extracting other data.

In [127]:
with open(f'{cr.calc.label}.txt') as of:
    st = 0
    for l in of:
        if st==0 and 'Phonon wavevector' in l :
            st=1
            print(f'\nk-point: {[float(v) for v in l.split()[-3:]]}')

        if 0<st:
            #print('>>>', l.strip())
            pass
            
        tok = [] 
        if st > 0:
            tok = l[1:].split()

        if st==2 and len(tok)==6:
            dt.append(float(l.split()[4]))
            if len(dt)==9:
                dt=array(dt).reshape((3,3))
                print('\nDielectric tensor:\n', dt)
                st=0
            
        if st==0 and 'Dielectric tensor' in l:
            dt=[]
            st=2
            continue

            
        if st==31 and len(tok)==6:
            ec.append(float(l.split()[4]))
            if len(ec)==18:
                ec=array(ec).reshape((-1,3))
                print(ec)
                st=0                  

        if st==3 and '(from':
            print(f'\nEffective charges {l.strip()}:')
            st=31
            continue
            
        if st==0 and 'Effective charges' in l:
            st=3
            ec=[]
            continue
            
            
        if st==11 :
            if tok:
                frqs += [float(v) for v in tok]
            else :
                print('Frequencies (cm^-1):\n', frqs)
                st=1
            
        if st==1 :
            if 'cm-1' in l:
                st=11
                frqs = []
            if 'direction' in tok[:1]:
                print(f'\nDirection: {[float(v) for v in l.split()[-3:]]}')
        
        if st==1 and '== DATASET' in l:
            print()
            st=0




Dielectric tensor:
 [[10.49162258  0.          0.        ]
 [ 0.         10.49162258  0.        ]
 [ 0.          0.         10.49162258]]

Effective charges (from electric field response):
[[-2.17820736  0.         -0.        ]
 [ 2.09060374  0.          0.        ]
 [ 0.         -2.17820736 -0.        ]
 [ 0.          2.09060374  0.        ]
 [ 0.         -0.         -2.17820736]
 [ 0.          0.          2.09060374]]

Effective charges (from phonon response):
[[-2.1782074   0.          0.        ]
 [ 0.         -2.1782074   0.        ]
 [-0.         -0.         -2.1782074 ]
 [ 2.09060389  0.          0.        ]
 [ 0.          2.09060389  0.        ]
 [ 0.          0.          2.09060389]]

k-point: [0.0, 0.0, 0.0]
Frequencies (cm^-1):
 [-0.09953807, -0.09953763, -0.0995363, 365.5292, 365.5292, 365.5292]

Direction: [1.0, 0.0, 0.0]
Frequencies (cm^-1):
 [-0.09953807, -0.09953763, 2.600481, 365.5292, 365.5292, 396.8464]

Direction: [0.0, 1.0, 0.0]
Frequencies (cm^-1):
 [-0.09953807,